In [10]:
import sympy as sp
from sympy import symbols, sin, cos, exp, log, sqrt
import random

# Define the symbolic variable
x = symbols('x')

# Function to generate random constants as rational numbers with at most two decimal places
def random_constant():
    numerator = random.randint(-1000, 1000)
    denominator = 100  # To get at most two decimal places
    # Simplify the fraction
    return sp.Rational(numerator, denominator)

# Function to check if an expression is constant (does not contain variable x)
def is_constant(expr):
    return expr.is_Number and not expr.has(x)

# List of unary functions
unary_functions = [sin, cos, log, sqrt]

# Function to generate random expressions, ensuring 'x' is included when needed
def random_expression(depth, include_variable):
    if depth <= 0:
        if include_variable:
            # Must include 'x' at this leaf
            return x
        else:
            # Return a random constant
            return random_constant()
    else:
        # Decide whether to use an operator or a unary function
        if random.choice(['operator', 'function']) == 'function':
            # Choose a unary function
            func = random.choice(unary_functions)
            # Generate the argument for the function
            arg = random_expression(depth - 1, include_variable)
            # Handle domain restrictions for log and sqrt
            if func in [log, sqrt]:
                # Ensure the argument is positive
                max_attempts = 5
                for _ in range(max_attempts):
                    arg = random_expression(depth - 1, include_variable)
                    # Check if the argument is positive
                    if (arg.is_real and arg.is_positive) or arg.has(x):
                        break
                else:
                    # If no positive argument is found, default to x + constant
                    arg = x + random_constant()
            return func(arg)
        else:
            # Choose an operator
            op = random.choice(['+', '-', '*', '**'])

            # Decide which side should include 'x'
            if include_variable:
                include_in_left = random.choice([True, False])
                left = random_expression(depth - 1, include_variable=include_in_left)
                right = random_expression(depth - 1, include_variable=not include_in_left)
            else:
                left = random_expression(depth - 1, include_variable=False)
                right = random_expression(depth - 1, include_variable=False)

            # Handle operations carefully to prevent large constants
            if op == '**':
                # Prevent exponentiation of two constants
                if is_constant(left) and is_constant(right):
                    left = x
                # Limit exponent to small positive integers
                if is_constant(right):
                    right = sp.Integer(random.randint(1, 3))
                # Limit base to x or small constant
                if is_constant(left):
                    left_value = abs(left.evalf())
                    if left_value > 5:
                        left = sp.Rational(random.randint(-500, 500), 100)
                return left ** right
            elif op == '*':
                # Prevent multiplication of two constants
                if is_constant(left) and is_constant(right):
                    # Replace one side with 'x'
                    left = x
                return left * right
            elif op == '+':
                return left + right
            elif op == '-':
                return left - right

# Set the desired depth of the expression tree
max_depth = 3

# Generate a random expression, ensuring 'x' is included
expr = random_expression(max_depth, include_variable=True)

# Simplify the expression
simplified_expr = sp.simplify(expr)

# Convert any floats to rationals
rational_expr = sp.nsimplify(simplified_expr, rational=True)

# Function to remove tiny constants from the expression
def remove_small_constants(expr, threshold=1e-10):
    if expr.is_Number:
        if abs(expr.evalf()) < threshold:
            return sympy.Integer(0)
        else:
            return expr
    elif expr.is_Symbol:
        return expr
    else:
        args = [remove_small_constants(arg, threshold) for arg in expr.args]
        return expr.func(*args)

# Apply the function to the expression
final_expr = remove_small_constants(rational_expr)

# Display the expressions
print(f"Random Expression: {expr}")
print(f"Simplified Expression: {simplified_expr}")
print(f"Final Expression: {final_expr}")


Random Expression: cos(cos(log(x)))
Simplified Expression: cos(cos(log(x)))
Final Expression: cos(cos(log(x)))
